## Notebook that will evaluate the KNN model

In [1]:
#importing libraires to use
import sklearn.preprocessing as skl_pre
import sklearn.neighbors as skl_nb
import sklearn.model_selection as skl_ms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#need to get the processing python file in another directory
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from process_data import process_data

In [2]:
#getting the percentage splits of our training and testing data
split_prec = {
    'train': 0.8, 
    'test': 0.2,
}
#whatever scaler we will use
scaler = skl_pre.StandardScaler()

#get the training and testing data
X_train, X_test, Y_train, Y_test = process_data(split_prec, scaler,is_random=False)
print(X_train)
print(X_test)
print(Y_train)
print(Y_test)

Dropped columns: ['snow']
New columns: ['good_weather' 'is_day' 'temp_fahrenheit']
Split: "train" 	[Size: 1280] 	[Prec: 0.8]
	X: (1280, 17)
	Y: (1280,)
Split: "test" 	[Size: 320] 	[Prec: 0.2]
	X: (320, 17)
	Y: (320,)
      hour_of_day  day_of_week     month   holiday   weekday  summertime  \
0        0.558988    -1.017343 -0.709870 -0.184189  0.643712    0.751527   
1       -1.625091    -1.516270  0.152211 -0.184189  0.643712    0.751527   
2        1.141408    -0.518416  1.014292 -0.184189  0.643712    0.751527   
3       -1.625091     1.477292 -0.135149 -0.184189 -1.553491    0.751527   
4        1.287014    -1.017343  0.726932 -0.184189  0.643712    0.751527   
...           ...          ...       ...       ...       ...         ...   
1275     1.723829     0.978365  0.439572 -0.184189 -1.553491    0.751527   
1276    -1.625091     0.978365  1.589013 -0.184189 -1.553491   -1.330625   
1277    -0.605854    -0.518416  0.726932 -0.184189  0.643712    0.751527   
1278    -0.023433    -0

### Training we will only be working with X_train from here on down
Now that we have our training and test data we will run a KNN model on the training data with k-fold validation for hyperparameter tunning (what is a good value of k, distance measure to use, ect...). Then once we have a decent value of k we will "retrain" the model on the entire training data set and use that model on the test data which has never been seen and use the error from that as an estimation of $E_{new}$

In [3]:
#returns a trained KNN classifier with the specifications passed
def get_fitted_KNN(X,Y, k = 1, metric = "minkowski"):
    classifier = skl_nb.KNeighborsClassifier(n_neighbors=k, metric=metric).fit(X,Y)
    return classifier

#this will run k_folds cross validation with the passed hyperparameters
#returns the approximation of the accuracy of the model
def run_k_folds(df = pd.concat([X_train,Y_train]), folds = 25, k = 1, metric = "minkowski"):
    #setting up our kfolds cross validation
    kf = skl_ms.KFold(folds, shuffle= True)
    #getting the column list so we can separate into X and Y
    cols = df.columns.tolist()
    #only the x columns
    xs = cols.remove("increase_stock")
    #will be the mean accuracy of the model. This is what we return
    score = 0
    print(df)
    #loop through all the training and test data
    for i, (train_index, test_index) in enumerate(kf.split(df)):
        #get the specific training data
        train = df[cols].loc[train_index]
        #get the specific testing data
        test = df[cols].loc[test_index]
        #make a KNN classifier and fit it 
        classifier = get_fitted_KNN(X = train[xs], Y=train["increase_stock"], k=k, metric=metric)
        #get the score using the k folds test data set
        score += classifier.score(test[xs], test["increase_stock"])
    return score/folds

In [4]:
print(run_k_folds())

      hour_of_day  day_of_week     month   holiday   weekday  summertime  \
0        0.558988    -1.017343 -0.709870 -0.184189  0.643712    0.751527   
1       -1.625091    -1.516270  0.152211 -0.184189  0.643712    0.751527   
2        1.141408    -0.518416  1.014292 -0.184189  0.643712    0.751527   
3       -1.625091     1.477292 -0.135149 -0.184189 -1.553491    0.751527   
4        1.287014    -1.017343  0.726932 -0.184189  0.643712    0.751527   
...           ...          ...       ...       ...       ...         ...   
1275          NaN          NaN       NaN       NaN       NaN         NaN   
1276          NaN          NaN       NaN       NaN       NaN         NaN   
1277          NaN          NaN       NaN       NaN       NaN         NaN   
1278          NaN          NaN       NaN       NaN       NaN         NaN   
1279          NaN          NaN       NaN       NaN       NaN         NaN   

          temp       dew  humidity    precip  snowdepth  windspeed  \
0     1.073206  0

KeyError: '[1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1369, 1370, 1371, 1372, 1373, 1374, 1375, 1376, 1377, 1378, 1379, 1380, 1381, 1382, 1383, 1384, 1385, 1386, 1387, 1388, 1389, 1390, 1391, 1392, 1393, 1394, 1395, 1397, 1398, 1399, 1400, 1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408, 1409, 1410, 1411, 1412, 1413, 1414, 1415, 1416, 1417, 1418, 1419, 1420, 1421, 1422, 1423, 1424, 1425, 1426, 1428, 1429, 1430, 1431, 1432, 1433, 1434, 1435, 1436, 1437, 1438, 1439, 1440, 1441, 1442, 1443, 1444, 1445, 1447, 1448, 1451, 1452, 1453